In [9]:
import pandas as pd
import numpy as np
import re
from shapely.geometry import Point, Polygon
import geopandas as gpd
import fiona
fiona.supported_drivers 

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'rw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [10]:
def to_float(i):
    i = float(i)
    return i

In [11]:
def transform(row):
    input = row.split(',')
    # print(input)
    size_list = len(input)
    # print(f'largo de lista: {size_list}')
    if size_list ==1:
        coor = []
    elif len(input) ==2:
        try:
            pre_punto = map(to_float,input)
        except ValueError as e:
            print(e)
            coor=list(pre_punto)
    elif len(input) >2:
        not_point = re.split(r",0\\n\\n|,0|0\s-|,",row)
        if ("" in not_point):
            not_point.remove("")
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
        else:
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
        return coor



In [12]:
def to_matrix(listado):
    n = len(listado)
    c = 2
    f =  n/c
    f = int(f)
    vector = np.array(listado)
    matriz = vector.reshape(f,c)
    return matriz

In [13]:
def geometry_apply(x):

    n = x
    if len(n) == 2:
            geometry = Point(n)
    elif len(n) >5:
        listado = to_matrix(n)
        geometry = Polygon(listado)
    else:
        geometry = None
    return geometry

    

In [14]:
def typeshape(x):
    n=x
    if int(n) == 2:
        label = "Point"
    else:
        label = "Polygon"
    return label    

In [15]:
def main(file_input):
    try:
        sitios= pd.read_csv(file_input,delimiter='\t',index_col='ID del Sitio')
        sitiosbd = sitios.copy()
        sitiosbd['info_geo'] = sitiosbd['Coordenadas areas']
        sitiosbd['info_geo']=sitiosbd['info_geo'].astype(str)
        sitiosbd['info_geo']= sitiosbd['info_geo'].apply(transform)
        desechados=sitiosbd[sitiosbd['info_geo'].isnull()]
        desechados.to_csv('./errors.csv')
        sitiosbd_depurate=sitiosbd[sitiosbd['info_geo'].notna()]
        sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
        sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)
        sitiosbd_depurate=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].notna()]
        sitiosbd_depurate['type_shape'] = sitiosbd_depurate['n_vertices'].apply(typeshape)
        sitiosbd_polygon = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Polygon']
        sitiosbd_point = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Point']
        gdf_points_sitios = gpd.GeoDataFrame(sitiosbd_point,geometry='type_geometry',crs='CRS84')
        gdf_polygons_sitios = gpd.GeoDataFrame(sitiosbd_polygon,geometry='type_geometry',crs='CRS84')
        gdf_points_sitios.to_file("./points.geojson",crs='CRS84', driver='GeoJSON')
        gdf_polygons_sitios.to_file("./polygons.geojson",crs='CRS84', driver='GeoJSON')
    except FileNotFoundError as e:
        print('Por favor ejecute el script en el mismo directorio donde se encuentre el archivo Sitio.txt {e}')
    

In [16]:
if __name__ == "__main__":
    main("./Sitios.txt")

C:\Users\JMELENDEZ\AppData\Local\Temp\ipykernel_36668\3887139163.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
C:\Users\JMELENDEZ\AppData\Local\Temp\ipykernel_36668\3887139163.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)


NameError: name 'GeoJSON' is not defined